# Logs

>  Convert xes event logs to csv format

In [1]:
#| default_exp logs

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Convert to pandas dataframe 

In [4]:
#| export
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter
import gzip
import json

In [ ]:
#| export
def convert_to_df(path):
    log = xes_importer.apply(bpi_2012)
    df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
    return df

In [7]:
bpi_2012 = "../data/logs/xes/BPI_Challenge_2012.xes.gz"
convert_to_df(bpi_2012)

parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
4,NaN,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
...,...,...,...,...,...,...,...
262195,112,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 22:51:17.423000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262196,112,SCHEDULE,W_Afhandelen leads,2012-02-29 22:52:01.287000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262197,11169,START,W_Afhandelen leads,2012-03-01 08:26:46.736000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262198,11169,COMPLETE,A_DECLINED,2012-03-01 08:27:37.118000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000


## Sample Conversion for PDC and BINET Logs

In [ ]:
#|eval: false
import warnings
warnings.filterwarnings('ignore')

### PDC 2020

#### Training Logs

In [ ]:
#|eval: false
for fn in progress_bar(glob.glob('data/orig/unzipped/PDC2020_training/*')):
    new_fn=fn.split('.')[0]+'.csv.gz'
    new_fn='data/csv/'+"/".join(new_fn.split('/')[3:])
    log = xes_importer.apply(fn)
    df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
    df.to_csv(new_fn,index=False,compression='gzip')


#### Groundtruth Logs

In [ ]:
#|eval: false
for fn in progress_bar(glob.glob('data/orig/unzipped/PDC2020_ground_truth/*')):
    new_fn=fn.split('.')[0]+'.csv.gz'
    new_fn='data/csv/'+"/".join(new_fn.split('/')[3:])
    log = xes_importer.apply(fn)
    df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
    df.to_csv(new_fn,index=False,compression='gzip')


### PDC 2021

#### Training Logs

In [ ]:
#|eval: false
for fn in progress_bar(glob.glob('data/orig/unzipped/PDC2021_training/*')):
    new_fn=fn.split('.')[0]+'.csv.gz'
    new_fn='data/csv/'+"/".join(new_fn.split('/')[3:])
    log = xes_importer.apply(fn)
    df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
    df.to_csv(new_fn,index=False,compression='gzip')

#### Groundtruth Logs


In [ ]:
#|eval: false
for fn in progress_bar(glob.glob('data/orig/unzipped/PDC2021_ground_truth/*')):
    new_fn=fn.split('.')[0]+'.csv.gz'
    new_fn='data/csv/'+"/".join(new_fn.split('/')[3:])
    log = xes_importer.apply(fn)
    df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
    df.to_csv(new_fn,index=False,compression='gzip')


### BINET Logs

In [ ]:
#|eval: false
def load_binet_data (path):
    with gzip.open(path, "r") as f:
        data = f.read()
        j = json.loads (data.decode('utf-8'))
        
    event_df = pd.DataFrame({'attributes': [], 'name': [], 'timestamp': [],'timestamp_end': [], 'event_id': [], 'trace_id': []})
    truth_df = pd.DataFrame({'case': [], 'anomalies': []})
    res=[]
    res=pd.DataFrame()
    for case in (j['cases']):
        trace = pd.DataFrame.from_dict(case['events'])
        trace['anomaly']= case['attributes']['label'] if type(case['attributes']['label'])==str else case['attributes']['label']['anomaly']
        trace['trace_id']=case['id']
        res=res.append(trace)
    res=pd.concat([res.drop(['attributes'], axis=1), res['attributes'].apply(pd.Series)], axis=1)
    return res
     

In [ ]:
#|eval: false
for fn in progress_bar((list(glob.glob('data/orig/unzipped/binet_logs/*-0.3-*')))):
    new_fn=fn[:-7]+'csv.gz'
    new_fn='data/csv/'+"/".join(new_fn.split('/')[3:])
    df=load_binet_data(fn)
    df.to_csv(new_fn,index=False,compression='gzip')

## Export -

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()